# Backtest Class

Copy the code to Backtes.py after finishing

In [42]:
import pandas as pd
import numpy as np
import math
import matplotlib.dates as dt
import matplotlib.pyplot as plt

In [9]:
class Backtest:
    """
    Des: Backtest using historal data and transcation view
    
    Input:
        NA is not allowed
        Monthly frequence
        date(increasing): %Y/%m/%d
        history data: time series of history price of the assets
        transcation view: view of long, short or close a position
        risk-free rate: time series of risk-free rate
        initial_asset(=100): the value of initital asset
        frequency: Y-year; Q-quarter; M-month; D-day
    """

    def __init__(self, df_date, df_history,df_rf, df_view,freq,initial_asset=100):
        self.df_date = df_date
        self.initial_asset = initial_asset
        self.df_rf = df_rf
        self.df_history = df_history
        self.df_view = df_view
        self.freq = freq
        
        ###  Calculate for Net Value of Asset with the transcation views  ###
        ###  Main Idea: use the return return of 2 adjacent days to solve this problem  ###
        
        # price obtained by simply view times history price
        ay_view = np.array(df_view)
        ay_history = np.array(df_history)
        df_price_each = pd.DataFrame(ay_view*ay_history)
        df_price = df_price_each.sum(axis=1)

        # bool value that wether the view changes or not 
        df_view_cg = ((abs(df_view-df_view.shift(1))).sum(axis=1))!=0
        ay_view_cg = np.array(df_view_cg)

        # the price of asset when position changes
        ay_view_close = np.array(df_view.shift(1))*np.repeat(ay_view_cg,3).reshape(6,3) + np.array(df_view.shift(0))*np.repeat(ay_view_cg==0,3).reshape(6,3)
        df_price_each_close = pd.DataFrame(ay_view_close*ay_history)
        df_price_each_close.iloc[0,:] = df_price_each.iloc[0,:]
        ay_price_each_close = np.array(df_price_each_close)

        # the return rate of each day puls 1
        # return between 2 adjacent days = the price of asset when position changes - the price obtained by simply view times history price
        df_r = ((df_price_each_close-df_price_each.shift(1)).sum(axis=1))/df_price.shift(1) + 1
        df_r[0] = 1

        # the true value of assets each month
        df_asset = df_r.cumprod()*100

        # time series of net value with initital asset = initial_asset
        self.df_asset = df_asset
        
        # log return of asserts each month
        df_log_return = np.log(df_asset / df_asset.shift(1))
        self.df_log_return = df_log_return
        
        if(self.freq = 'Y') :
            Times = 1
        if(self.freq = 'Q') :
            Times = 4
        if(self.freq = 'M') :
            Times = 12
        if(self.freq = 'D') :
            Times = 252

    # Maximum retracement of the portfolio
    def maxdown(self):
        df_asset = self.df_asset
        md = ((df_asset.cummax()-df_asset)/df_asset).max()
        return md
    
    # Annualized return of the portfolio
    def annualized_return(self):
        df_asset = self.df_asset
        start = 0;
        lst_annualized_return = []
        while(True):
            try:
                df_asset[start+Times]
            except:
                break
            if(len(df_asset[start:start+Times+1])<Times):
                break
            Log_return = math.log(df_asset[start+Times]/df_asset[start])
            lst_annualized_return.append(Log_return)
            start = start + Times
        return lst_annualized_return
    
    # Annualized volatility of the portfolio
    def annualized_volatility(self):
        YV = np.std(self.annualized_return()) * np.sqrt(Times)
        return YV
    
    # Sharpe ratio of the portfolio
    def sharpe(self):
        ER = mean(self.annualized_return())
        RF = mean(list(self.df_rf))
        YV = self.annualized_volatility()
        return (ER - RF) / YV
    
    # Net value of the portfolio
    def net_value(self):
        return self.df_asset
        
    def output_info(self):
        """
        Des: Print the info of the backtest results

        """
        printf("The maxdown is %.4f", %(self.maxdown())+"\n")
        printf("The annualized return is %.4f",%(self.annualized_return())+"\n")
        printf("The annualized volatility is %.4f",%(self.annualized_volatility())+"\n")
        printf("The annualized sharpe ratio is %.4f",%(self.sharpe())+"\n")
        printf("The net value is %.4f",%(self.net_value())+"\n")
        
        
    def gen_graph(self):
        """
        Des: gen the graph of backtest (using R)
        """
        
        
    def save_graph(self, path):
        """
        Des: Save the graph of backtest
        Input: 
            path: the path for saving graph
        """
        
    

# Data for Testing

In [15]:
df_date = pd.DataFrame({"date":["2020/01/01","2020/01/02","2020/01/03","2020/01/02","2020/01/04","2020/01/05","2020/01/06"]})
df_view = pd.DataFrame({"v1":[1,0,1,1,1,0],"v2":[1,1,2,0,0,1],"v3":[1,0,0,1,1,-1]})
df_history = pd.DataFrame({"p1":[1,2,2,3,5,6],"p2":[1,1,1,1,1,2],"p3":[2,3,4,5,6,7]})
initial_asset = 100

In [16]:
ay_view = np.array(df_view)
ay_history = np.array(df_history)
df_price_each = pd.DataFrame(ay_view*ay_history)
df_price = df_price_each.sum(axis=1)

# bool value that wether the view changes or not 
df_view_cg = ((abs(df_view-df_view.shift(1))).sum(axis=1))!=0
ay_view_cg = np.array(df_view_cg)

# the price of asset when position changes
ay_view_close = np.array(df_view.shift(1))*np.repeat(ay_view_cg,3).reshape(6,3) + np.array(df_view.shift(0))*np.repeat(ay_view_cg==0,3).reshape(6,3)
df_price_each_close = pd.DataFrame(ay_view_close*ay_history)
df_price_each_close.iloc[0,:] = df_price_each.iloc[0,:]
ay_price_each_close = np.array(df_price_each_close)

# the return rate of each day puls 1
# return between 2 adjacent days = the price of asset when position changes - the price obtained by simply view times history price
df_r = ((df_price_each_close-df_price_each.shift(1)).sum(axis=1))/df_price.shift(1) + 1
df_r[0] = 1

# the true value of assets each month
df_asset = df_r.cumprod()*100

In [61]:
list(df_asset)

[100.0, 150.0, 150.0, 187.5, 257.8125, 304.6875]

In [21]:
df_asset[0:5]

0    100.0000
1    150.0000
2    150.0000
3    187.5000
4    257.8125
dtype: float64

In [22]:
df_asset[0+5-1]

257.8125

In [54]:
start=0
Times=1
lst_annualized_return = []
while(True):
    try:
        df_asset[start+Times]
    except:
        break
    if(len(df_asset[start:start+Times+1])<Times):
        break
    Log_return = math.log(df_asset[start+Times]/df_asset[start])
    lst_annualized_return.append(Log_return)
    start = start + Times

In [55]:
df_asset

0    100.0000
1    150.0000
2    150.0000
3    187.5000
4    257.8125
5    304.6875
dtype: float64

In [57]:
lst_annualized_return

[0.4054651081081644,
 0.0,
 0.22314355131420976,
 0.3184537311185346,
 0.16705408466316624]

In [58]:
np.std(lst_annualized_return)

0.13803169229420945

In [40]:
df_asset[start:start+Times]

0    100.0
dtype: float64

In [6]:
df_history

,p1,p2,p3
0,1,1,2
1,2,1,3
2,2,1,4
3,3,1,5
4,5,1,6
5,6,2,7


In [7]:
df_view.join(df_history)

,v1,v2,v3,p1,p2,p3
0,1,1,1,1,1,2
1,0,1,0,2,1,3
2,1,2,0,2,1,4
3,1,0,1,3,1,5
4,1,0,1,5,1,6
5,0,1,-1,6,2,7


In [53]:
df_view-df_view.shift(1)

,v1,v2,v3
0,NaN,NaN,NaN
1,-1.0,0.0,-1.0
2,1.0,1.0,0.0
3,0.0,-2.0,1.0
4,0.0,0.0,0.0
5,-1.0,1.0,-2.0
